# GRPC Inference

### Setup

In [1]:
!pip install grpcio grpcio-tools opencv-python-headless

In [2]:
grpc_host = 'modelmesh-serving'
grpc_port = 8033
model_name = 'resnet'
image_path = 'images/bee.jpeg'
input_name = "gpu_0/data_0"

In [3]:
import grpc
import grpc_predict_v2_pb2
import grpc_predict_v2_pb2_grpc


channel = grpc.insecure_channel(f"{grpc_host}:{grpc_port}")
stub = grpc_predict_v2_pb2_grpc.GRPCInferenceServiceStub(channel)

request = grpc_predict_v2_pb2.ModelMetadataRequest(name=model_name)
response = stub.ModelMetadata(request)
response

name: "resnet__isvc-e06002a97f"
versions: "1"
platform: "OpenVINO"
inputs {
  name: "gpu_0/data_0"
  datatype: "FP32"
  shape: -1
  shape: 3
  shape: 224
  shape: 224
}
outputs {
  name: "gpu_0/softmax_1"
  datatype: "FP32"
  shape: 1
  shape: 1000
}

### Preprocess image

In [4]:
import numpy as np
import cv2

def preprocess(img_data):
    mean_vec = np.array([0.485, 0.456, 0.406])
    stddev_vec = np.array([0.229, 0.224, 0.225])
    norm_img_data = np.zeros(img_data.shape).astype('float32')
    for i in range(img_data.shape[0]):
        # for each pixel in each channel, divide the value by 255 to get value between [0, 1] and then normalize
        norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]
    return norm_img_data

def getJpeg(path, size):
    with open(path, mode='rb') as file:
        content = file.read()

    img = np.frombuffer(content, dtype=np.uint8)
    img = cv2.imdecode(img, cv2.IMREAD_COLOR)  # BGR format
    # format of data is HWC
    # add image preprocessing if needed by the model
    img = cv2.resize(img, (224, 224))
    img = img.astype('float32')
    #convert to NCHW
    img = img.transpose(2,0,1)
    # normalize to adjust to model training dataset
    img = preprocess(img)
    img = img.reshape(1,3,size,size)
    print(path, img.shape, "; data range:",np.amin(img),":",np.amax(img))
    return img



In [5]:
img = getJpeg(image_path, 224)

images/bee.jpeg (1, 3, 224, 224) ; data range: -2.117904 : 2.64


### Request

In [6]:
import time

# request content building
inputs = []
inputs.append(grpc_predict_v2_pb2.ModelInferRequest().InferInputTensor())
inputs[0].name = input_name
inputs[0].datatype = "FP32"
inputs[0].shape.extend([1, 3, 224, 224])
arr = img.flatten()
inputs[0].contents.fp32_contents.extend(arr)

# request building
request = grpc_predict_v2_pb2.ModelInferRequest()
request.model_name = model_name
request.inputs.extend(inputs)

t1 = time.time()
response = stub.ModelInfer(request)
t2 = time.time()
inference_time = t2-t1
print(f"Request took {inference_time}")

Request took 0.07062077522277832


In [8]:
import classes

result_arr = np.frombuffer(response.raw_output_contents[0], dtype=np.float32)
max = np.argmax(result_arr)
print("Highest score: class {} with score {}".format(max, arr[max]))
print("Detected class name: {}".format(classes.imagenet_classes[max]))


Highest score: class 309 with score -1.741159439086914
Detected class name: bee
